# Transport problem

In this tutorial we investigate the transport problem.

## Exercise 1

We consider the convervative formulation of a linear transport problem: the only unknown is the concentration $c$.

Let $\Omega = [0, 1]^2$ be the domain of interest with boundary $\partial \Omega$ and outward unit normal ${\nu}$. 
We define also $(0, T)$ the time interval, being $T$ the final time.

Given a flux $q = [1, 0]^\top$ , we want to solve the following problem: find $c$ such that
$$
\partial_t c + \nabla \cdot (qc) = 0
\quad \text{in } \Omega \times (0, T)
$$
with boundary conditions are set on the inflow of the domain:
$$ c = 1 \quad \text{ on } \partial_{left} \Omega \times (0, T)$$
and initial condition for the contration
$$ c(x, 0) = 0 \quad \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

Before creating the grid we import NumPy, the SciPy sparse library and PorePy.

In [ ]:
import numpy as np
import scipy.sparse as sps

import porepy as pp

We specify number of cells in each dimension and the physical size of the domain. Then we create a Cartesian grid and compute geometric properties such as face centers, cell volumes etc.

In [ ]:
dim = 2
N = [40] * dim
phys_dims = [1] * dim

sd = pp.CartGrid(N, phys_dims)
sd.compute_geometry()

We define now the data related to the temporal discretization and the flux $q$, since the solver we consider later will need the flux exchange over each face and due to the position of the face normals in the `CartGrid` the flux can be easily defined.

In [ ]:
delta_t = 0.1
num_steps = 10

q = sd.face_normals.T @ [1, 0, 0]

We declare the darcy velocity and the boundary conditions

In [ ]:
# define outflow and inflow type boundary conditions, left and right boundary
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]

outflow = np.isclose(b_face_centers[0, :], 1)
inflow = np.isclose(b_face_centers[0, :], 0)

# define the labels and values for the boundary faces
labels = np.array(["neu"] * b_faces.size)
bc_val = np.zeros(sd.num_faces)

labels[np.logical_or(inflow, outflow)] = "dir"
bc_val[b_faces[inflow]] = 1

bc = pp.BoundaryCondition(sd, b_faces, labels)

parameters = {"darcy_flux": q, "bc": bc, "bc_values": bc_val}

Once all the data are created we group them in a dictionary, where the keyword `"transport"` represent the physical process considered and ensures that the discretization objects use the correct parameters. Note that the call to `initialize_default_data assignes` default values to the transport parameters which are not specified.

In [ ]:
transport_key = "transport"
transport_data = pp.initialize_default_data(sd, {}, transport_key, parameters)

In this section we present all the approaches to solve the problem.

In [ ]:
# create the upwind and mass matrices
upwind = pp.Upwind(transport_key)

# discretize and get the matrices
upwind.discretize(sd, transport_data)

U, b_upwind = upwind.assemble_matrix_rhs(sd, transport_data)
M = sps.diags(sd.cell_volumes)

Let us do now the temporal loop and consider an approximation with the implicit Euler scheme.

In [ ]:
# Initial condition and exporter
c = np.zeros(sd.num_cells)
save = pp.Exporter(sd, "ie_transport", folder_name="ex1")
save.write_vtu([("conc", c)], time_step=0)

# IE
S = M + delta_t * U
for i in np.arange(num_steps):
    c = sps.linalg.spsolve(S, M @ c - delta_t * b_upwind)
    save.write_vtu([("conc", c)], time_step=(i + 1) * delta_t)
    print(np.amax(c), np.amin(c))

# export the main pvd file
time = np.arange((num_steps + 1)) * delta_t
save.write_pvd(time)

The solution is now exported for each time step and can be visualzied with ParaView.

Let us consider now a temporal discretization done with the explicit Euler scheme, with a finer time step to avoid unphysical oscillations.

In [ ]:
delta_t = 0.04  # 0.04 0.02
num_steps = 50  # 25 50

save = pp.Exporter(sd, "ee_transport", folder_name="ex1")
# Initial condition and exporter
c = np.zeros(sd.num_cells)
save.write_vtu([("conc", c)], time_step=0)

# change the format of the matrix
M = M.tocsr()

# EE
S = M - delta_t * U
for i in np.arange(num_steps):
    c = sps.linalg.spsolve(M, S @ c - delta_t * b_upwind)
    save.write_vtu([("conc", c)], time_step=(i + 1) * delta_t)
    print(np.amax(c), np.amin(c))

# export the main pvd file
time = np.arange((num_steps + 1)) * delta_t
save.write_pvd(time)